In [1]:
import platform
print("python version " + platform.python_version())
import sys
import json
import os
os.chdir("/Users/chenry/code/cb_annotation_ontology_api/notebooks")
import re
import pandas as pd
from os.path import exists
from pathlib import Path
import logging
COBRAKBase_path = "/Users/chenry/code/cobrakbase"
BaseModule_path = "/Users/chenry/code/KBBaseModules"
ModelSEEDpy_path = "/Users/chenry/code/ModelSEEDpy"
ModelSEEDReconstruction_path = "/Users/chenry/code/KB-ModelSEEDReconstruction"
sys.path.append(COBRAKBase_path)
sys.path.append(BaseModule_path)
sys.path.append(ModelSEEDpy_path)
sys.path.append(ModelSEEDReconstruction_path+"/lib")
import cobrakbase
from modelseedpy import ModelSEEDBiochem

python version 3.9.13
cobrakbase 0.3.1


In [2]:
kbase_api = cobrakbase.KBaseCache()
template = kbase_api.get_from_ws("GramNegModelTemplateV5_test",12218)
biochem = ModelSEEDBiochem.get()
filter_df = [] 
no_form = {}
allowed = ["cpd11416"]
for compound in biochem.compounds:
    if str(compound.formula) == "nan" and compound.id not in allowed:
        no_form[compound.id+"_0"] = 1
print(len(no_form))
count = 0
for reaction in biochem.reactions:
    if re.search("MI", reaction.status) != None:
        filter_df.append({"id":reaction.id,"reason":"MI"})
        #if reaction.id+"_c" in template.reactions:
            #print("MI:",reaction.id)
    elif re.search("CI", reaction.status) != None and reaction.id+"_c" not in template.reactions:
        filter_df.append({"id":reaction.id,"reason":"CI"})
        #if reaction.id+"_c" in template.reactions:
            #print("CI:",reaction.id)
    elif reaction.is_obsolete == True:
        filter_df.append({"id":reaction.id,"reason":"OBS"})
        #if reaction.id+"_c" in template.reactions:
            #print("OBS:",reaction.id)
    elif len(reaction.metabolites) == 0:
        filter_df.append({"id":reaction.id,"reason":"Empty"})
        #if reaction.id+"_c" in template.reactions:
            #print("Empty:",reaction.id)
    else:
        for met in reaction.metabolites:
            if met.id in no_form:
                if reaction.id+"_c" not in template.reactions:
                    filter_df.append({"id":reaction.id,"reason":"NOFORM"})
                    count += 1
                #if reaction.id+"_c" in template.reactions:
                    #print("NOFORM:",reaction.id,met.id)
                break            
df = pd.DataFrame.from_records(filter_df)
df.to_csv("FilteredReactions.csv")

3469


In [3]:
os.getcwd()

'/Users/chenry/code/cb_annotation_ontology_api/notebooks'